# Task-1

**Assignment 1: Object Detection for a Toy Manufacturer (10 Points)**

A well-known toy manufacturer has approached you with a challenge related to the development of a sorting line. Among several contractors, you have been assigned the task of detecting the manufacturer’s products on a conveyor belt. You previously submitted a proposal for a Proof of Concept (PoC), which has now been accepted.

Your main task is to train an object detection model capable of recognizing the client’s products. You may disregard the integration into the actual conveyor system for now.

However, the model must be robust to varying orientations of the objects — the toys may appear from any angle except from underneath, as they will never be placed upside down.

Each group will receive a bag of objects that need to be detected by the model.

Your work is divided into the following sub-tasks:

####  1.1. Design a Data Collection Strategy

Plan and implement your own data acquisition process. Create a custom dataset including proper annotations for object detection.

Tip: For object annotation, you can use tools like Label Studio, which allows you to easily annotate images with bounding boxes and export the data in common formats (e.g., COCO or Pascal VOC).

## Data Collection and Labeling Strategy

### 1. Real Image Collection

To train a robust model for toy detection on a conveyor belt, we captured real images of the client’s toy products—**donkey** and **tortoise**—from **all possible angles**, including **upside down**. This simulates real-world conditions where toys can land on the conveyor in unpredictable orientations.

- **Background**: All images were taken against a **black background** to replicate the conveyor belt.
- **Dataset Breakdown**:
  - 53 images of **donkey only**
  - 123 images of **tortoise only**
  - 23 images with **both donkey and tortoise**

**Total real images: 199**

---

### 2. Labeling Real Images

All real images were labeled using **Label Studio** and exported in **Pascal VOC format**, generating XML files with object class names and bounding boxes.

---

### 3. Synthetic Image Generation

To enhance the dataset and improve model performance, we generated **300 synthetic images** using a custom **Python script**. Here's the approach:

- Croped both toys tightly

#### a. Centered Placement (100 images)
- 50 donkey and 50 tortoise images placed at the **center** of 512×512 black canvases.

#### b. Random Placement (100 images)
- 50 donkey and 50 tortoise images placed at **random positions** within 512×512 canvases.

#### c. Dual Object Placement (100 images)
- Each image contains both **donkey and tortoise**, placed **randomly without overlapping**.

All toys were rotated randomly in steps of 30 degrees to simulate various orientations.

Bounding boxes were calculated for each placement, and labels were saved in **VOC XML format**.

---

### 4. Python Script Summary

We developed a script that:
- Loads and rotates toy images
- Places them on a black 512×512 background
- Generates corresponding VOC-style XML labels

This automation ensures consistent, varied, and accurate labeling.

---

###  Final Dataset Summary

- **Real labeled images**: 199
- **Synthetic labeled images**: 300
- **Total**: 499 images for training, validation, and testing
---

### 1.2. Train and Evaluate an Object Detection Model

Choose an object detection approach of your choice (the techniques covered in the lecture provide a solid foundation). Train your model and evaluate it using a self-created test set consisting of four representative images.

Use a standard evaluation metric to assess performance.

Note: While the test set consists of only a few images and is intended primarily for symbolicevaluation, make sure your training process is based on a meaningful dataset split as discussed in class. The main assessment of your model’s performance will take place during the live demonstration.

### Yolo - Model Training

Converting VOC to YOLO labels and spliting into train, validation and test sets. 

In [1]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

# === Define paths and class map ===
base_dir = r"C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data"
splits = ['train', 'val', 'test']
class_map = {"donkey": 0, "tortoise": 1} 

for split in splits:
    annot_dir = os.path.join(base_dir, split, 'annotations')
    label_dir = os.path.join(base_dir, split, 'labels')
    os.makedirs(label_dir, exist_ok=True)

    print(f"\n🔄 Converting {split} annotations to YOLO format...")
    xml_files = [f for f in os.listdir(annot_dir) if f.endswith(".xml")]

    for xml_file in tqdm(xml_files):
        xml_path = os.path.join(annot_dir, xml_file)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        img_filename = root.find("filename").text
        size = root.find("size")
        img_w = int(size.find("width").text)
        img_h = int(size.find("height").text)

        yolo_lines = []
        for obj in root.findall("object"):
            cls_name = obj.find("name").text.strip().lower()
            if cls_name not in class_map:
                continue
            cls_id = class_map[cls_name]

            bbox = obj.find("bndbox")
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)

            x_center = ((xmin + xmax) / 2) / img_w
            y_center = ((ymin + ymax) / 2) / img_h
            bw = (xmax - xmin) / img_w
            bh = (ymax - ymin) / img_h

            yolo_lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}")

        label_filename = os.path.splitext(xml_file)[0] + ".txt"
        label_path = os.path.join(label_dir, label_filename)

        with open(label_path, "w") as f:
            f.write("\n".join(yolo_lines))



🔄 Converting train annotations to YOLO format...


100%|███████████████████████████████████████████████████████████████████████████████| 349/349 [00:02<00:00, 128.40it/s]



🔄 Converting val annotations to YOLO format...


100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [00:01<00:00, 98.38it/s]



🔄 Converting test annotations to YOLO format...


100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:00<00:00, 85.17it/s]


In [2]:
import os
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
# Ignore all UserWarnings globally
warnings.filterwarnings("ignore", category=UserWarning)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('ERROR')
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0))

CUDA available: True
Device count: 1
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
from ultralytics import YOLO

from ultralytics import settings

# Update a setting
settings.update({"mlflow": True})

# Reset settings to default values
settings.reset()

# Load a pre-trained YOLOv8 model
model = YOLO("yolov8s.pt")

# Train the model with mlflow logging disabled
model.train(
    data=r"C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\toy_data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    project="toy_model_results",
    name="yolov8s_toy",
    workers=0,
)


New https://pypi.org/project/ultralytics/8.3.123 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.103  Python-3.9.16 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=C:\Sem_5\Portfolio1\Task-1\Toy_Data\toy_data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=0, project=toy_model_results, name=yolov8s_toy12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cla

train: Scanning C:\Sem_5\Portfolio1\Task-1\Toy_Data\train\labels.cache... 349 images, 0 backgrounds, 0 corrupt: 100%|██


albumentations: __init__() got an unexpected keyword argument 'quality_range'


val: Scanning C:\Sem_5\Portfolio1\Task-1\Toy_Data\val\labels.cache... 99 images, 0 backgrounds, 0 corrupt: 100%|███████


Plotting labels to toy_model_results\yolov8s_toy12\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


...

                   all         99        131      0.999      0.993      0.994      0.974



50 epochs completed in 1.971 hours.
Optimizer stripped from toy_model_results\yolov8s_toy12\weights\last.pt, 22.5MB
Optimizer stripped from toy_model_results\yolov8s_toy12\weights\best.pt, 22.5MB

Validating toy_model_results\yolov8s_toy12\weights\best.pt...
Ultralytics 8.3.103  Python-3.9.16 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 72 layers, 11,126,358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:25<0


                   all         99        131      0.999      0.993      0.993      0.978
                donkey         55         55      0.999          1      0.995      0.972
              tortoise         75         76      0.999      0.987      0.991      0.983
Speed: 1.4ms preprocess, 20.0ms inference, 0.0ms loss, 5.3ms postprocess per image
Results saved to toy_model_results\yolov8s_toy12
MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002CE9671E520>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0

After training the YOLOv8 model for 50 epochs, we achieved a mAP
@ 0.5:0.95 of **0.98**, indicating excellent detection performance.


### Testing and Save predictions

Please check the test folder for prediction on testset.

In [1]:
import os
from ultralytics import YOLO
from PIL import Image

# Load the best trained model
model = YOLO("toy_model_results/yolov8s_toy11/weights/best.pt")

# Directory containing test images
test_img_dir = r"C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images"
output_dir = r"C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\predictions"
os.makedirs(output_dir, exist_ok=True)

# List to store inference-only times (in milliseconds)
inference_times_ms = []

# Run inference on each image
for img_file in os.listdir(test_img_dir):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(test_img_dir, img_file)

        # Run full inference and get results
        results = model(img_path)

        # Access inference-only time from speed dictionary
        inference_ms = results[0].speed.get('inference', 0)
        inference_times_ms.append(inference_ms)

        # Save prediction image
        results[0].save(filename=os.path.join(output_dir, img_file))

# Calculate mean inference time in ms and FPS
mean_inference_ms = sum(inference_times_ms) / len(inference_times_ms)
mean_inference_s = mean_inference_ms / 1000
mean_fps = 1 / mean_inference_s if mean_inference_s > 0 else float('inf')

print(f"Mean inference-only time per image: {mean_inference_ms:.2f} ms")
print(f"Mean inference-only speed: {mean_fps:.2f} FPS")



image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_001.jpg: 640x480 1 donkey, 211.6ms
Speed: 6.8ms preprocess, 211.6ms inference, 426.6ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_011.jpg: 480x640 1 donkey, 160.5ms
Speed: 8.1ms preprocess, 160.5ms inference, 6.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_027.jpg: 480x640 1 tortoise, 36.0ms
Speed: 8.0ms preprocess, 36.0ms inference, 5.9ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_035.jpg: 480x640 1 tortoise, 29.6ms
Speed: 6.0ms preprocess, 29.6ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_045.jpg: 640x480 1 tortoise, 28

Speed: 11.8ms preprocess, 40.9ms inference, 6.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_303.jpg: 640x640 1 donkey, 73.5ms
Speed: 21.7ms preprocess, 73.5ms inference, 12.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_331.jpg: 640x640 1 donkey, 69.6ms
Speed: 27.0ms preprocess, 69.6ms inference, 8.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_356.jpg: 640x640 1 tortoise, 27.8ms
Speed: 7.0ms preprocess, 27.8ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Toy_Data\test\images\image_386.jpg: 640x640 1 tortoise, 44.0ms
Speed: 9.7ms preprocess, 44.0ms inference, 11.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Sem_5\Po

### 1.3. Demonstrate the Model in a Realistic Scenario

The client has deliberately requested only a small test set. Instead, they require a finaldemonstration showing object  etection on a continuous data stream. You are free to decide (depending on your chosen approach) whether this demonstration is based on video analysis or a live demo. In both cases, the results must be representative of the intended real-world application.

Requirements for the demo:

• Detected objects must be enclosed in visible bounding boxes with class prediction

• The model must detect each object from all possible perspectives (except from underneath, as described above)

• Detection must be continuous

### Inference 

To test our trained model in a real-world scenario, we created a video simulating a conveyor belt with toy objects. The goal was to run inference on this video using our YOLOv8 model.

Run the code below to perform inference on the video.

Make sure to check the **output video** to see the detection results — it will be saved automatically after processing.


In [2]:
import cv2
import pyautogui
from ultralytics import YOLO

# Load the trained model
model = YOLO("toy_model_results/yolov8s_toy11/weights/best.pt")

# Open the video file
video_path = r"C:\Sem_5\Portfolio1\Task-1\Task-1-YOLO-Important\Inference_Video.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Output video writer
out = cv2.VideoWriter("output_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Create display window
cv2.namedWindow("YOLOv8 Inference", cv2.WINDOW_NORMAL)
screen_width, screen_height = pyautogui.size()
cv2.resizeWindow("YOLOv8 Inference", screen_width, screen_height)

frame_num = 0
while frame_num < frame_count:
    ret, frame = cap.read()
    if not ret:
        print(f"Frame {frame_num} couldn't be read. Stopping.")
        break

    # Inference
    results = model(frame)
    annotated_frame = results[0].plot()

    # Save output
    out.write(annotated_frame)

    # Display
    cv2.imshow("YOLOv8 Inference", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

    frame_num += 1

# Cleanup
cap.release()
out.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 349.3ms
Speed: 16.2ms preprocess, 349.3ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 82.0ms
Speed: 10.8ms preprocess, 82.0ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 83.8ms
Speed: 13.6ms preprocess, 83.8ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 56.5ms
Speed: 6.5ms preprocess, 56.5ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.3ms
Speed: 12.4ms preprocess, 65.3ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.8ms
Speed: 5.9ms preprocess, 38.8ms inference, 5.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.5ms
Speed: 7.0ms preprocess, 38.5ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.3ms
Speed: 8.2ms preprocess, 38


0: 384x640 1 donkey, 1 tortoise, 11.8ms
Speed: 2.4ms preprocess, 11.8ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 12.6ms
Speed: 1.9ms preprocess, 12.6ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 11.9ms
Speed: 2.7ms preprocess, 11.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 11.9ms
Speed: 2.6ms preprocess, 11.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 14.6ms
Speed: 2.5ms preprocess, 14.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 11.8ms
Speed: 2.6ms preprocess, 11.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 14.7ms
Speed: 4.8ms preprocess, 14.7ms inference, 7.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 14.7ms
Speed: 2.3ms preprocess, 14.7ms inference, 1.5ms post


0: 384x640 1 donkey, 1 tortoise, 16.8ms
Speed: 2.2ms preprocess, 16.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.2ms
Speed: 2.9ms preprocess, 17.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.2ms
Speed: 2.4ms preprocess, 17.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.1ms
Speed: 3.1ms preprocess, 17.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.0ms
Speed: 3.5ms preprocess, 17.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 16.9ms
Speed: 2.0ms preprocess, 16.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 16.9ms
Speed: 2.5ms preprocess, 16.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 


0: 384x640 1 donkey, 1 tortoise, 17.8ms
Speed: 2.0ms preprocess, 17.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.7ms
Speed: 2.0ms preprocess, 17.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 18.4ms
Speed: 3.5ms preprocess, 18.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 18.0ms
Speed: 2.5ms preprocess, 18.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.8ms
Speed: 2.3ms preprocess, 17.8ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 18.0ms
Speed: 3.4ms preprocess, 18.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 18.1ms
Speed: 3.3ms preprocess, 18.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 


0: 384x640 1 donkey, 18.0ms
Speed: 2.3ms preprocess, 18.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 17.9ms
Speed: 2.7ms preprocess, 17.9ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 17.7ms
Speed: 2.9ms preprocess, 17.7ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 17.8ms
Speed: 2.3ms preprocess, 17.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.8ms
Speed: 3.1ms preprocess, 17.8ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.9ms
Speed: 2.6ms preprocess, 17.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.9ms
Speed: 2.8ms preprocess, 17.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 17.7ms
Speed: 4.2ms preprocess, 17.7ms inference

...


0: 384x640 1 donkey, 1 tortoise, 19.4ms
Speed: 2.4ms preprocess, 19.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 19.6ms
Speed: 2.1ms preprocess, 19.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 19.6ms
Speed: 2.3ms preprocess, 19.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 19.7ms
Speed: 2.4ms preprocess, 19.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 20.0ms
Speed: 2.7ms preprocess, 20.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 20.3ms
Speed: 2.7ms preprocess, 20.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 20.0ms
Speed: 2.2ms preprocess, 20.0ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 


0: 384x640 1 donkey, 1 tortoise, 19.5ms
Speed: 2.4ms preprocess, 19.5ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 19.3ms
Speed: 2.2ms preprocess, 19.3ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 19.4ms
Speed: 2.2ms preprocess, 19.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tortoise, 19.5ms
Speed: 3.0ms preprocess, 19.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tortoise, 19.3ms
Speed: 2.0ms preprocess, 19.3ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tortoise, 19.4ms
Speed: 2.2ms preprocess, 19.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tortoise, 19.4ms
Speed: 2.5ms preprocess, 19.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tortoise, 19.4ms
Speed: 2.0ms preprocess, 19.4ms inference, 


0: 384x640 1 donkey, 1 tortoise, 41.3ms
Speed: 4.9ms preprocess, 41.3ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 42.1ms
Speed: 5.5ms preprocess, 42.1ms inference, 14.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 41.2ms
Speed: 4.4ms preprocess, 41.2ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 55.4ms
Speed: 4.4ms preprocess, 55.4ms inference, 6.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 41.4ms
Speed: 9.7ms preprocess, 41.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 66.7ms
Speed: 11.0ms preprocess, 66.7ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 62.5ms
Speed: 11.6ms preprocess, 62.5ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoi

Speed: 4.1ms preprocess, 38.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 38.3ms
Speed: 4.2ms preprocess, 38.3ms inference, 9.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 donkey, 1 tortoise, 37.3ms
Speed: 7.2ms preprocess, 37.3ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 donkeys, 1 tortoise, 37.6ms
Speed: 9.8ms preprocess, 37.6ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 donkeys, 1 tortoise, 36.9ms
Speed: 4.6ms preprocess, 36.9ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 donkeys, 1 tortoise, 41.8ms
Speed: 15.6ms preprocess, 41.8ms inference, 7.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 donkeys, 1 tortoise, 37.4ms
Speed: 13.9ms preprocess, 37.4ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 donkeys, 1 tortoise, 81.6ms
Speed: 14.0ms preprocess, 8

Speed: 5.9ms preprocess, 85.9ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tortoise, 38.1ms
Speed: 8.6ms preprocess, 38.1ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tortoise, 38.4ms
Speed: 10.5ms preprocess, 38.4ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 tortoise, 38.6ms
Speed: 14.7ms preprocess, 38.6ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.9ms
Speed: 4.4ms preprocess, 37.9ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.2ms
Speed: 4.5ms preprocess, 38.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.9ms
Speed: 4.2ms preprocess, 37.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.8ms
Speed: 4.6ms preprocess, 37.8ms inference, 1.5ms postprocess per image at shape (1

### Using Webcam

In [ ]:
import cv2
import time
import pyautogui
from ultralytics import YOLO

# Load your trained 2-class model (donkey, tortoise)
model = YOLO("toy_model_results/yolov8s_toy11/weights/best.pt")

# Open webcam
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 384)

cv2.namedWindow("YOLOv8 - Toy Detection (Webcam)", cv2.WINDOW_NORMAL)

# Resize window to screen
screen_width, screen_height = pyautogui.size()
cv2.resizeWindow("YOLOv8 - Toy Detection (Webcam)", screen_width, screen_height)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run inference
    start_time = time.time()
    results = model(frame)
    inference_time = (time.time() - start_time) * 1000  # in ms

    result = results[0]
    annotated_frame = frame.copy()

    if result.boxes is not None:
        boxes = result.boxes
        for box in boxes:
            cls_id = int(box.cls.item())
            conf = box.conf.item()
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            if conf < 0.35:
                label = f"other {conf:.2f}"
                color = (0, 0, 255)  # Red
            else:
                class_name = model.model.names[cls_id].lower()

                if class_name == "donkey":
                    label = f"donkey {conf:.2f}"
                    color = (255, 0, 0)  # Blue
                elif class_name == "tortoise":
                    label = f"tortoise {conf:.2f}"
                    color = (0, 255, 0)  # Green
                else:
                    continue  # Skip anything unexpected

            # Draw box and label
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(annotated_frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Show inference time & FPS
    fps_label = f"Inference: {inference_time:.1f} ms ({1000 / inference_time:.2f} FPS)"
    cv2.putText(annotated_frame, fps_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.9, (255, 255, 0), 2)

    cv2.imshow("YOLOv8 - Toy Detection (Webcam)", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 (no detections), 22.6ms
Speed: 4.3ms preprocess, 22.6ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 36.3ms
Speed: 4.0ms preprocess, 36.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 78.3ms
Speed: 12.1ms preprocess, 78.3ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 60.4ms
Speed: 9.6ms preprocess, 60.4ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 62.0ms
Speed: 10.5ms preprocess, 62.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.2ms
Speed: 2.3ms preprocess, 37.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15.9ms
Speed: 1.5ms preprocess, 15.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.0ms
Speed: 5.7ms preprocess, 40.0ms


0: 384x640 (no detections), 12.7ms
Speed: 1.7ms preprocess, 12.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.6ms
Speed: 1.6ms preprocess, 12.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 13.1ms
Speed: 1.9ms preprocess, 13.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.5ms
Speed: 2.0ms preprocess, 12.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.6ms
Speed: 1.6ms preprocess, 12.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.4ms
Speed: 1.5ms preprocess, 12.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.2ms
Speed: 2.9ms preprocess, 12.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.4ms
Speed: 1.3ms preprocess, 12.4ms i


0: 384x640 (no detections), 12.2ms
Speed: 1.3ms preprocess, 12.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.1ms
Speed: 1.4ms preprocess, 12.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.0ms
Speed: 1.4ms preprocess, 12.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.8ms
Speed: 1.4ms preprocess, 11.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.2ms
Speed: 1.5ms preprocess, 12.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.9ms
Speed: 1.2ms preprocess, 11.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.1ms
Speed: 2.1ms preprocess, 12.1ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 11.9ms
Speed: 1.4ms preprocess, 11.9ms i


0: 384x640 (no detections), 12.5ms
Speed: 1.4ms preprocess, 12.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.6ms
Speed: 1.2ms preprocess, 12.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.6ms
Speed: 1.4ms preprocess, 12.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.8ms
Speed: 1.3ms preprocess, 12.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.5ms
Speed: 1.3ms preprocess, 12.5ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.5ms
Speed: 1.2ms preprocess, 12.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.9ms
Speed: 1.4ms preprocess, 12.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.7ms
Speed: 1.6ms preprocess, 12.7ms i


0: 384x640 (no detections), 48.4ms
Speed: 4.4ms preprocess, 48.4ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 44.5ms
Speed: 6.3ms preprocess, 44.5ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 45.7ms
Speed: 2.1ms preprocess, 45.7ms inference, 2.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 51.7ms
Speed: 5.7ms preprocess, 51.7ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 81.7ms
Speed: 2.7ms preprocess, 81.7ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 81.4ms
Speed: 4.7ms preprocess, 81.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 79.0ms
Speed: 7.3ms preprocess, 79.0ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 64.3ms
Speed: 6.9ms preprocess, 64.3ms i

...


0: 384x640 (no detections), 38.7ms
Speed: 8.4ms preprocess, 38.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 36.8ms
Speed: 6.1ms preprocess, 36.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 36.5ms
Speed: 5.3ms preprocess, 36.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.0ms
Speed: 3.5ms preprocess, 37.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 40.5ms
Speed: 10.8ms preprocess, 40.5ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 36.9ms
Speed: 8.8ms preprocess, 36.9ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 58.2ms
Speed: 7.8ms preprocess, 58.2ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 36.8ms
Speed: 5.5ms preprocess, 36.8ms 


0: 384x640 (no detections), 56.9ms
Speed: 3.0ms preprocess, 56.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 72.6ms
Speed: 10.5ms preprocess, 72.6ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 106.2ms
Speed: 7.7ms preprocess, 106.2ms inference, 7.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 83.0ms
Speed: 12.0ms preprocess, 83.0ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 54.7ms
Speed: 6.9ms preprocess, 54.7ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 65.9ms
Speed: 13.2ms preprocess, 65.9ms inference, 6.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 87.7ms
Speed: 11.2ms preprocess, 87.7ms inference, 6.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 88.6ms
Speed: 8.3ms preprocess, 88


0: 384x640 (no detections), 38.1ms
Speed: 3.1ms preprocess, 38.1ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.1ms
Speed: 3.4ms preprocess, 38.1ms inference, 6.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 78.0ms
Speed: 10.1ms preprocess, 78.0ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 76.1ms
Speed: 9.3ms preprocess, 76.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 66.3ms
Speed: 8.3ms preprocess, 66.3ms inference, 4.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 39.3ms
Speed: 10.8ms preprocess, 39.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 41.2ms
Speed: 5.1ms preprocess, 41.2ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.0ms
Speed: 6.3ms preprocess, 38.0ms


0: 384x640 (no detections), 37.9ms
Speed: 3.8ms preprocess, 37.9ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.8ms
Speed: 5.1ms preprocess, 37.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.0ms
Speed: 8.2ms preprocess, 38.0ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.9ms
Speed: 5.7ms preprocess, 37.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.9ms
Speed: 3.0ms preprocess, 37.9ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 38.7ms
Speed: 6.7ms preprocess, 38.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 47.4ms
Speed: 5.4ms preprocess, 47.4ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.9ms
Speed: 2.5ms preprocess, 37.9ms i


0: 384x640 (no detections), 59.2ms
Speed: 7.4ms preprocess, 59.2ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 37.0ms
Speed: 3.1ms preprocess, 37.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)



<h1 style="text-align:center;">The End</h1>
